In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


In [ ]:
events_df.columns

## Cantidad eventos total

In [6]:
df = events_df.groupby('person').event.count()

In [7]:
features_df = features_df.join(df, on='person')

In [8]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Quincenas

In [17]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [ ]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [ ]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

## Cant ev indiv

In [9]:
df = events_df.groupby('person').event.value_counts()

In [10]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [11]:
df = df.drop(columns=['lead'])

In [13]:
df['ch+cv'] = df.checkout + df.conversion
df['ch+cv-vp'] = df.checkout + df.conversion - df['viewed product']

In [14]:
features_df = features_df.join(df, on='person')

In [15]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

## Dia semana

In [18]:
events_df['weekday'] = events_df['timestamp'].dt.day_name()

In [19]:
ct = pd.crosstab(events_df.person, events_df.weekday)
ct = ct.astype('bool')

In [20]:
from mlxtend.frequent_patterns import apriori 

frequent_itemsets = apriori(ct, min_support=0.4, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.414098,(Thursday),1
1,0.404466,(Tuesday),1
2,0.416982,(Wednesday),1


In [23]:
result = columna_por_persona_new(ct)

In [28]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38829

In [29]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38829

In [30]:
d = {'person': lista_personas, 'weekday_support': lista_support}
df = pd.DataFrame(data=d)

In [31]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,ch+cv,ch+cv-vp,weekday_support
0,4886f805,9,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,NaN,0.0
1,ad93850f,65,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,NaN,NaN,0.0
2,0297fc1e,567,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,NaN,NaN,0.0


## Hora

In [32]:
events_df['hora'] = events_df['timestamp'].dt.hour

In [33]:
ct = pd.crosstab(events_df.person, events_df.hora)
ct = ct.astype('bool')

In [34]:
from mlxtend.frequent_patterns import apriori 

frequent_itemsets = apriori(ct, min_support=0.15, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.211002,(0),1
1,0.198202,(1),1
2,0.161477,(2),1
3,0.179582,(12),1
4,0.209817,(13),1
5,0.226351,(14),1
6,0.224162,(15),1
7,0.232275,(16),1
8,0.231296,(17),1
9,0.226223,(18),1


In [35]:
result = columna_por_persona_new(ct)

In [36]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38829

In [37]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38829

In [38]:
d = {'person': lista_personas, 'hora_support': lista_support}
df = pd.DataFrame(data=d)

In [39]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,ch+cv,ch+cv-vp,weekday_support,hora_support
0,4886f805,9,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,NaN,0.0,0.211002
1,ad93850f,65,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,NaN,NaN,0.0,0.211002
2,0297fc1e,567,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,NaN,NaN,0.0,0.211002


## Supports 

In [64]:
## url

ct = pd.crosstab(events_df.person, events_df.url)
ct = ct.astype('bool')

In [65]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.518247,(/),1
1,0.080857,(/comprar/iphone),1
2,0.012823,(/comprar/iphone/),1
3,0.099386,(/comprar/iphone/6),1
4,0.016961,(/comprar/iphone/6-plus),1


In [66]:
def columna_por_persona(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        lista_value = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > 0 :
                lista_value.append(value)
        values = frozenset(lista_value)        
        diccionario[person] = values                
    return diccionario

In [68]:
result = columna_por_persona_new(ct)

In [25]:
#Algoritmo para asignarle el support a la persona segun sus eventos(Toma todos los ev realizados por la persona)
#Result es lo que devuelve storage por persona
def asignar_support(result, frequent_itemsets):
    lista_support = list()
    for value in result.values() :
        #print(value)
        support_value = obtener_support(value, frequent_itemsets)
        #print(support_value)
        lista_support.append(support_value)
    return lista_support   

In [26]:
def obtener_support(nupla, frequent_itemsets):
    if( len(frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values) == 0 ):
        return 0;
    support_value = frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values[0]
    return support_value 

In [69]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

31896

In [27]:
def lista_personas_f(result):
    lista_personas = list()
    for key in result.keys():
        lista_personas.append(key)
    return lista_personas 

In [70]:
lista_personas = lista_personas_f(result)
len(lista_personas)

31896

In [71]:
d = {'person': lista_personas, 'url_support': lista_support}
df = pd.DataFrame(data=d)

In [72]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,ch+cv,ch+cv-vp,weekday_support,hora_support,url_support
0,4886f805,9,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,NaN,0.0,0.211002,NaN
1,ad93850f,65,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,NaN,NaN,0.0,0.211002,0.518247
2,0297fc1e,567,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,NaN,NaN,0.0,0.211002,0.518247


In [ ]:
#sku (no, mucho por procesar)

ct = pd.crosstab(events_df.person, events_df.sku)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona(ct)
result.shape

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'weekday_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
# model

ct = pd.crosstab(events_df.person, events_df.model)
ct = ct.astype('bool')

In [ ]:
ct.shape

In [ ]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.1, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona(ct)

In [ ]:
len(result)

In [ ]:
features_df.shape

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'model_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
len(result)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'model_ind_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
# Condition

ct = pd.crosstab(events_df.person, events_df.condition)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'condition_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
# Storage

ct = pd.crosstab(events_df.person, events_df.storage)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.5, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'storage_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
#color

ct = pd.crosstab(events_df.person, events_df.color)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.2, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [22]:
def columna_por_persona_new(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        num_max = 0
        lista = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > num_max :
                value_max = value
                num_max = crosstab.loc[person, value]
        lista.append(value_max)
        values = frozenset( lista ) 
        diccionario[person] = values                
    return diccionario

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
len(result)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'color_3_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
#features_df = features_df.drop(columns=['color_2_support'])

In [ ]:
#Channel

ct = pd.crosstab(events_df.person, events_df.channel)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.25, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'channel_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
events_df.columns

In [ ]:
#Device_type?

ct = pd.crosstab(events_df.person, events_df.device_type)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'dev_type_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
#Operatng_system_version

ct = pd.crosstab(events_df.person, events_df.operating_system_version)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'op_sys_ver_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
#Screen resolution

ct = pd.crosstab(events_df.person, events_df.screen_resolution)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'screen_res_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Labels y separacion de df

In [40]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [73]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 18)

In [74]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 17)

In [75]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [76]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [77]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## XGBoost

In [46]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
#from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.model_selection import cross_validate
from sklearn import metrics
#from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

def modelfit(alg, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=True, show_stdv=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train, eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(X_test)
        
    #Print model report:
    print( "\nModel Report" )
    print( "AUC Score (Train): %f" % metrics.roc_auc_score(y_test, dtrain_predictions) )
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [ ]:
xg_r = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.8, learning_rate = 0.2,
                max_depth = 5, alpha = 20, gamma=0, n_estimators = 35, eval_metric='auc')

In [ ]:
modelfit(xg_r, X_train, y_train)

In [ ]:
#Tunear max_depth y min_depth

param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
gsearch1.best_params_, gsearch1.best_score_

In [47]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


In [48]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3, 0.5],
        'colsample_bylevel': [0.7, 0.8, 0.9],
        'max_depth': [3, 4, 5, 7, 8, 9, 11, 13, 15],
        'base_score': [0.1, 0.2, 0.3],
        'n_estimators': [35, 40, 50, 60, 70, 80, 90],
        'booster':['gbtree', 'dart']
        }

In [49]:
xg_gs = xgb.XGBRegressor(objective ='binary:logistic',
                        learning_rate = 0.2,
                        alpha = 20, eval_metric='auc')

In [78]:
folds = 10
param_comb = 7

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xg_gs, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, 
                                   cv=skf.split(train_df.drop(columns=['person']),labels), verbose=3, random_state=1001,return_train_score=True)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(train_df.drop(columns=['person']), labels)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   15.6s
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:   27.6s finished



 Time taken: 0 hours 0 minutes and 28.42 seconds.


In [79]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)

print('\n Best hyperparameters:')
print(random_search.best_params_)


 All results:
{'mean_fit_time': array([3.2142704 , 1.81366873, 0.4749121 , 0.27122838, 1.13483555,
       0.42380888, 2.10047181]), 'std_fit_time': array([0.0928412 , 0.15758562, 0.04007365, 0.03314709, 0.03534874,
       0.01672415, 0.24969361]), 'mean_score_time': array([0.01353288, 0.00733716, 0.00747848, 0.00911872, 0.00754106,
       0.01211216, 0.0093534 ]), 'std_score_time': array([0.00124456, 0.00044476, 0.00254068, 0.00251753, 0.00154802,
       0.00242075, 0.0014027 ]), 'param_n_estimators': masked_array(data=[70, 70, 35, 40, 70, 50, 70],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 1, 10, 1, 5, 5, 1],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[5, 11, 4, 4, 5, 7, 15],
             mask=[False, False, False, False, False, False, False],
       

In [80]:
preds_grid_search = random_search.predict(X_test)

In [81]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds_grid_search)

0.8253198427284603

In [82]:
features_df.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_seh', 'cant_s_p', 'cant_sp', 'cant_vp',
       'cant_vs', 'ch+cv', 'ch+cv-vp', 'weekday_support', 'hora_support',
       'url_support'],
      dtype='object')

In [54]:
#Parametros con los que obtuvimos 0.82

xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 8, alpha = 20, gamma=10, n_estimators = 35, eval_metric='auc')

In [55]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [56]:
preds = xg_reg.predict(X_test)

In [57]:
roc_auc_score(y_test,preds)

0.8187511952572193

## Importancia con Random Forest

In [86]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
to_predict.fillna(0, inplace=True)

In [87]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(train_df.drop(columns=['person']).columns)

In [88]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [61]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [90]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictions)

0.7727779560534308

In [63]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cant_eventos_totales Importance: 0.14
Variable: cant_vp              Importance: 0.12
Variable: cant_bl              Importance: 0.09
Variable: cant_ch              Importance: 0.09
Variable: hora_support         Importance: 0.09
Variable: cant_ach             Importance: 0.08
Variable: cant_gl              Importance: 0.08
Variable: cant_seh             Importance: 0.07
Variable: cant_s_p             Importance: 0.07
Variable: cant_vs              Importance: 0.07
Variable: weekday_support      Importance: 0.03
Variable: cant_sp              Importance: 0.02
Variable: ch+cv-vp             Importance: 0.02
Variable: cant_cv              Importance: 0.01
Variable: ch+cv                Importance: 0.01


In [ ]:
features_df.columns

In [ ]:
features_df = features_df.drop(columns=['condition_support', 'model_support', 'cant_sp', 'dev_type_support', 'q3'])

In [ ]:
preds_rf = rf.predict(to_predict.drop(columns=['person']))

In [ ]:
preds_rf

In [83]:
train_df.fillna(0, inplace=True)

In [92]:
from sklearn import metrics
def score(x1,x2):
    return metrics.roc_auc_score(x1,x2)

# defining feature importance function based on above logic
def feat_imp(m, x, y, small_good = True): 
    score_list = {} 
    score_list['original'] = score(y, m.predict(x)) 
    imp = {} 
    for i in range(len(x.columns)): 
        rand_idx = np.random.permutation(len(x)) # randomization
        new_coli = x.values[rand_idx, i] 
        new_x = x.copy()            
        new_x[x.columns[i]] = new_coli 
        score_list[x.columns[i]] = score(y, m.predict(new_x)) 
        imp[x.columns[i]] = score_list['original'] - score_list[x.columns[i]] # comparison with benchmark
    if small_good: 
             return sorted(imp.items(), key=lambda x: x[1]) 
    else: return sorted(imp.items(), key=lambda x: x[1], reverse=True)

In [93]:
feat_imp(rf, train_df.drop(columns=['person']), labels)

[('cant_cv', 0.0019273669107437996),
 ('cant_sp', 0.0019521658071929293),
 ('ch+cv', 0.0022880580028473707),
 ('cant_ach', 0.005540975748007715),
 ('url_support', 0.005693367180874764),
 ('weekday_support', 0.006512284310491001),
 ('ch+cv-vp', 0.006554270834947751),
 ('cant_s_p', 0.008456506721165158),
 ('cant_seh', 0.00880286095125915),
 ('cant_gl', 0.013976364658915386),
 ('cant_vs', 0.015314121200178055),
 ('hora_support', 0.019456035099295277),
 ('cant_bl', 0.025683076745941835),
 ('cant_vp', 0.03738976115562853),
 ('cant_eventos_totales', 0.08493068487023758),
 ('cant_ch', 0.23857930554233187)]

## Prediccion

In [ ]:
preds_posta = random_search.predict(to_predict.drop(columns=['person']))

In [ ]:
preds_posta

In [ ]:
#encamble

ensamble = predictions*0.2 + preds_grid_search*0.8
roc_auc_score(y_test, ensamble)

In [ ]:
ensamble = preds_rf*0.2 + preds_posta*0.8
ensamble

In [ ]:
features_df.columns

## Genero CSV a publicar

In [ ]:
to_publish = pd.DataFrame()

In [ ]:
to_publish['person'] = to_predict.person
to_publish.shape

In [ ]:
to_publish['label'] = ensamble
to_publish.shape

In [ ]:
to_publish.head()

In [ ]:
to_publish.loc[ to_publish.label > 1 ]

In [ ]:
to_publish.loc[ to_publish.label < 0 ]

In [ ]:
to_publish.to_csv('2_12_6.csv', index=False)